In [1]:
import os 
import numpy as np
import matplotlib.pylab as plt
from itertools import zip_longest
# import torch
# from torch.utils.data import Dataset, DataLoader
import pandas as pd
import json
import collections
# import phycv
# from phycv import PST, PST_GPU,VEVID,VEVID_GPU
import matplotlib.image as mpimg
from PIL import Image
from io import BytesIO
import cv2
import tqdm
# import torchvision.transforms as transforms
import copy
from PIL import Image
from tqdm import tqdm
import shutil
tqdm.pandas()

## PST Preprocesser

In [2]:
class DataProcesser:
    def __init__(self,input_directory="../bdd100k/images/100k/",output_directory="../bdd100k_preprocessed/images/",labels_path = '../bdd100k/labels/',train=True,mode=2,df=None):
        if train:
            self.mode = "train"
        else:
            self.mode = "val"
        self.input_directory=os.path.join(input_directory,self.mode)
        self.output_directory=os.path.join(output_directory,self.mode)
        self.labels_path=labels_path
        if df is None:
            if mode==2:
                self.df = self.open_dataframe2()
            elif mode==1:
                self.df = self.open_dataframe()
        else:
            self.df = df
        
    def process_images(self,mode=2,morph_flag=1):
        def process(name):
            if mode==2:
                return name+".jpg"
            else:
                return name
        def helper(name,time):
            if time=="night":
                self.process_image_pst_vevid(os.path.join(self.input_directory,name),self.output_directory,name,morph_flag=morph_flag)
            else:
                self.process_image_pst_vevid(os.path.join(self.input_directory,name),self.output_directory,name,b=None,G=None,morph_flag=morph_flag)
        self.df.progress_apply(lambda x: helper(process(x["name"]), x["attributes.timeofday"]), axis=1)
    
    def open_dataframe(self):
        with open(self.labels_path+'bdd100k_labels_images_'+self.mode+'.json') as data_file:
            data = json.load(data_file)
        df = pd.json_normalize(data)
        return df
    
    def open_dataframe2(self):
        data=[]
        path = os.path.join(self.labels_path,self.mode)
        for f in os.listdir(path):
            with open(os.path.join(path,f)) as data_file:
                data_ = json.load(data_file)
            data.append(data_)
        df = pd.json_normalize(data)
        return df
    
    @staticmethod
    def process_image_pst_vevid(img_file=None,output_path=None,output_name=None,S=0.4, W=20, sigma_LPF=0.1, thresh_min=0.0,thresh_max=0.8,morph_flag=1,b = 0.2, G = 10):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if b is not None:
            assert(G is not None)
            vevid_gpu = VEVID_GPU(device=device)
            original_image = vevid_gpu.run_lite(img_file,b,G)
        else:
            original_image = transforms.ToTensor()(mpimg.imread(img_file))
        if output_path:
            if not os.path.exists(output_path):
                os.makedirs(output_path)
        pst_gpu = PST_GPU(device=device)
        pst_gpu.load_img(img_array=original_image)
        pst_gpu.init_kernel(S,W)
        pst_gpu.apply_kernel(sigma_LPF,thresh_min,thresh_max,morph_flag)
        pst_output_gpu_torch=pst_gpu.pst_output
        pst_output_gpu = pst_output_gpu_torch.cpu().numpy()
        if morph_flag==0:
            shape = pst_output_gpu.shape
            pst_output_gpu = ((np.argsort(np.argsort(pst_output_gpu.reshape((-1,))))+1)/pst_output_gpu.size).reshape(shape)
            fn = lambda x: x**5
            pst_output_gpu = fn(pst_output_gpu)
        pst_gpu_result = Image.fromarray((pst_output_gpu * 255).astype(np.uint8))
        if output_path:
            pst_gpu_result.save(os.path.join(output_path, output_name))
        return pst_gpu_result
    
    def conditional_copy(self,weather_condition=None,scene_condition=None,time_of_day=None):
        if not os.path.exists(self.output_directory):
            os.makedirs(self.output_directory)
        tmp = self.df
        if weather_condition is not None:
            tmp = tmp[tmp["attributes.weather"]==weather_condition]
        if scene_condition is not None:
            tmp = tmp[tmp["attributes.scene"]==scene_condition]
        if time_of_day is not None:
            tmp = tmp[tmp["attributes.timeofday"]==time_of_day]
        extension = os.path.splitext(os.listdir(self.input_directory)[0])[1]
        def copy(file):
            name = os.path.join( self.input_directory, file+extension )
            if os.path.isfile( name ) :
                shutil.copy( name, self.output_directory)
            else:
                print('file does not exist', name)
        tmp["name"].progress_apply(lambda x: copy(x))

In [39]:
#Change labels path to where the label json files are (not including train/ or val/)
train_processer = DataProcesser(train=True,labels_path='./data2/zwt/bdd/bdd100k/labels/100k',output_directory="../bdd100k_preprocessed/images/")
train_processer.process_images(morph_flag=1)

100%|██████████| 70000/70000 [6:40:55<00:00,  2.91it/s]      


In [40]:
val_processer = DataProcesser(train=False,labels_path='./data2/zwt/bdd/bdd100k/labels/100k',output_directory="../bdd100k_preprocessed/images/")
val_processer.process_images(morph_flag=1)

100%|██████████| 10000/10000 [13:40<00:00, 12.19it/s]


## Dark/Fog Images

In [3]:
def get_night_foggy_data(input_directories,labels_path = 'G:/UCLA/Lane Detection Project/Lane Detection/bdd100k_preprocessed/det_annotations/data2/zwt/bdd/bdd100k/labels/100k'):
    train_df = DataProcesser(train=True,labels_path=labels_path,output_directory="").df
    val_df = DataProcesser(train=False,labels_path=labels_path,output_directory="").df
    for dir in input_directories:
        for t in [True,False]:
            for condition in ["night","foggy"]:
                if dir.endswith('/'):
                    dir = dir[:-1]
                head,tail = os.path.split(dir)
                new_dir = os.path.join(head,tail+"_"+condition)
                if t:
                    processer = DataProcesser(train=t,labels_path=labels_path,input_directory=dir,output_directory=new_dir,df=train_df)
                else:
                    processer = DataProcesser(train=t,labels_path=labels_path,input_directory=dir,output_directory=new_dir,df=val_df)
                if condition=="night":
                    processer.conditional_copy(time_of_day="night")
                else:
                    processer.conditional_copy(weather_condition="foggy")

In [4]:
#Put in a list of directories as well as the label path to generate folders with _night and _foggy containing the night and foggy images
get_night_foggy_data(["G:/UCLA/Lane Detection Project/Lane Detection/bdd100k_preprocessed_analog/images","G:/UCLA/Lane Detection Project/Lane Detection/bdd100k_preprocessed/images","G:/UCLA/Lane Detection Project/Lane Detection/bdd100k_preprocessed/ll_seg_annotations/bdd_lane_gt","G:/UCLA/Lane Detection Project/Lane Detection/bdd100k_preprocessed/da_seg_annotations/bdd_seg_gt","G:/UCLA/Lane Detection Project/Lane Detection/bdd100k/images/100k"])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 543.09it/s]


In [5]:
get_night_foggy_data(['G:/UCLA/Lane Detection Project/Lane Detection/bdd100k_preprocessed/det_annotations/data2/zwt/bdd/bdd100k/labels/100k'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 1071.97it/s]
